<h1>Only to be executed when hosted on colab.research.google.com</h1>

In [1]:
!pip install pyLDAvis
!python -m spacy download de_core_news_sm
import os

!git clone https://github.com/YStrehlow/nlp-ws19.git

os.chdir('nlp-ws19')
https://github.com/Ystrehlow/nlp-ws19

SyntaxError: invalid syntax (<ipython-input-1-d144f4156c75>, line 8)

<h1>###################################################</h1>

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

import spacy
from spacy import displacy

import itertools
from tqdm import tqdm_notebook as tqdm

import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier

from sklearn.metrics import confusion_matrix, accuracy_score

from plot_cm import plot_confusion_matrix

from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

C:\Users\vravindran\AppData\Local\Continuum\anaconda3\envs\nlp\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Dataset

#### Read the .csv file into the variable "data"

In [2]:
data = pd.read_csv("articles.csv")

#### Print the first 10 rows

In [3]:
data[:10]

,id,char_count,content,title,url,category,content_preprocessed
0,337240,4894,'Wieder einmal will die Türkei Abgeordnete nic...,"'Erneuter Incirlik-Eklat Pokern um den ""Tornad...",'http://www.spiegel.de//politik/deutschland/in...,politik,'Türkei Abgeordnete Incirlik lassen. Bundesreg...
1,337259,469,'Stuttgart (dpa/lsw) - Ministerpräsident Winfr...,'Kretschmann empfängt Adelsfamilien zum gemein...,'http://www.sueddeutsche.de/news/politik/regie...,politik,'Stuttgart (dpa/lsw) - Ministerpräsident Winfr...
2,337262,372,'Cottbus (dpa/bb) - Der Braunkohlenausschuss d...,'Braunkohlenausschuss kommt nach Revierkonzept...,'http://www.sueddeutsche.de/news/wirtschaft/en...,wirtschaft,'Cottbus (dpa/bb) - Braunkohlenausschuss Land ...
3,337267,3404,'Frank Plasberg lässt die Erfolgsaussichten de...,"'""Hart aber fair"" mit Frank Plasberg Es muss n...",'http://www.sueddeutsche.de/medien/hart-aber-f...,medien,'Frank Plasberg lässt Erfolgsaussichten SPD-Ka...
4,337271,492,'Washington (dpa) - US-Präsident Donald Trump ...,'McMaster: Bericht über Trumps Geheimnis-Weite...,'http://www.sueddeutsche.de/news/politik/gehei...,politik,'Washington (dpa) - US-Präsident Donald Trump ...
5,337278,502,'New York (dpa) - UN-Generalsekretär António G...,'Vereinte Nationen verurteilen Nordkoreas Rake...,'http://www.sueddeutsche.de/news/politik/konfl...,politik,'New York (dpa) - UN-Generalsekretär António G...
6,337284,2638,'Düsseldorf – Die SPD will nach ihrer schweren...,'Nach NRW-Schlappe | SPD schließt große&lt;br ...,'http://www.bild.de/politik/inland/politik/nrw...,politik,'Düsseldorf – SPD schwer Niederlage Landtagswa...
7,337294,2317,'Nordkorea könnte Experten zufolge hinter der ...,'Experten: Nordkorea könnte hinter weltweiter ...,'http://www.donaukurier.de/nachrichten/wirtsch...,wirtschaft,'Nordkorea Experte zufolge jung weltweit Cyber...
8,337297,153,'Der Bundesgerichtshof (BGH) hat der Unterlass...,'BGH-Urteil - Darlehenskontogebühren bei Bausp...,'http://www.donaukurier.de/nachrichten/wirtsch...,wirtschaft,'Bundesgerichtshof (BGH) Unterlassungsklage Ve...
9,337298,2317,'Nordkorea könnte Experten zufolge hinter der ...,'Experten: Nordkorea könnte hinter weltweiter ...,'http://www.donaukurier.de/nachrichten/medien/...,medien,'Nordkorea Experte zufolge jung weltweit Cyber...


#### Investigate how many instances of each class are present in the data 

In [4]:
data['category'].value_counts()

sport         6366
politik       5110
wirtschaft    3341
kultur        1366
leben         1264
medien        1103
Name: category, dtype: int64

In [ ]:
data['category'].value_counts().plot(kind='bar')

In [ ]:
data.head()

## Classification Time

#### Load the german spacy word model

In [ ]:
nlp = spacy.load("de_core_news_sm")

#### Apply spacy pre-processing to content

In [ ]:
content = nlp.pipe(data['content'])

#### Read the stopwords file "stopwords.txt" line by line

In [ ]:
with open('stopwords.txt', 'r') as f:
    stopwords = f.readlines()

In [ ]:
stopwords[:10]

#### Remove whitespaces and newlines from the stopwords

In [ ]:
stopwords = [x.strip() for x in stopwords]

In [ ]:
def set_stop_word(sw):
    if sw.startswith("#") or sw.startswith("//"):
        return
    nlp.vocab[sw].is_stop = True
    sw = sw[0].upper() + sw[1:]
    nlp.vocab[sw].is_stop = True
    
for w in stopwords:
    set_stop_word(w)

#### Remove stopwords from content

In [ ]:
#this line takes really long to execute. Stopwords are removed and lemmas are extracted.
#the result from this line is already stored in data['content_preprocessed'] -> see next line

#content_without_stopwords=[]
#for t in content:
#    content_without_stopwords.append("".join(list("".join(tok.lemma_ + tok.whitespace_) for tok in t if not tok.is_stop)))

In [ ]:
content_without_stopwords = data['content_preprocessed']

In [ ]:
content_without_stopwords[3]

#### Split the data into two sets: one for training the classifier and the other one for testing the performance

In [ ]:
split = int((len(data)/100)*80)

train = data[:split]
test = data[split:].reset_index(drop=True)

#### Encode the texts with the TfidfVectorizer

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=1000, min_df=2)
train_x = tfidf_vectorizer.fit_transform(train['content_preprocessed']).toarray()
test_x = tfidf_vectorizer.transform(test['content_preprocessed']).toarray()

In [ ]:
tfidf_vectorizer.vocabulary_

In [ ]:
train_x[0]

#### Encode the labels with the LabelEncoder

In [ ]:
le = LabelEncoder()
train_y = le.fit_transform(train['category'])
test_y = le.transform(test['category'])

#### Random Forest Classifier (RandomForestClassifier)

In [ ]:
clf = RandomForestClassifier(10, random_state=2)
clf.fit(train_x, train_y)

predictions = clf.predict(test_x)
cnf_matrix = confusion_matrix(test_y, predictions)

In [ ]:
print(accuracy_score(predictions, test_y))
cf_matrix_plot = plot_confusion_matrix(cnf_matrix, classes=le.classes_, title='', normalize=True)

In [ ]:
cf_matrix_plot

In [ ]:
lda_texts = []
for text in data['content_preprocessed'][:1000]:
    lda_texts.append([tok.text for tok in nlp(text) if tok.is_alpha])

In [ ]:
common_dictionary = Dictionary(lda_texts)
common_corpus = [common_dictionary.doc2bow(text) for text in lda_texts]

lda = LdaModel(common_corpus, num_topics=10)

In [ ]:
vis = pyLDAvis.gensim.prepare(lda, common_corpus, dictionary=common_dictionary)

In [ ]:
vis

In [ ]:
politic_texts = []
for i, text in enumerate(data['content_preprocessed']):
    if i >= 1000:
        break
    if data['category'][i] == 'politik':
        politic_texts.append([tok.text for tok in nlp(text) if tok.is_alpha])

In [ ]:
politic_dictionary = Dictionary(politic_texts)
politic_corpus = [politic_dictionary.doc2bow(text) for text in politic_texts]

politic_lda = LdaModel(politic_corpus, num_topics=5)

In [ ]:
politic_vis = pyLDAvis.gensim.prepare(politic_lda, politic_corpus, dictionary=politic_dictionary)

In [ ]:
politic_vis